In [2]:
!pip install google-api-python-client

zsh:1: /usr/local/bin/pip: bad interpreter: /usr/local/opt/python@3.9/bin/python3.9: no such file or directory


In [3]:
import json
with open('secret.json') as f:
    secret = json.load(f)

In [4]:
from apiclient.discovery import build

In [5]:


DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
developerKey=DEVELOPER_KEY)

q = 'Python 自動化'
max_results = 30

response = youtube.search().list(
q=q,
part="id,snippet",
order='viewCount',
type='video',
maxResults=max_results
).execute()

In [6]:
items = response['items']
item = items[0]
item

{'kind': 'youtube#searchResult',
 'etag': 'eINzWSXP4VvYJqMPiFQHxScHDKs',
 'id': {'kind': 'youtube#video', 'videoId': 'bsq2YY-XAEo'},
 'snippet': {'publishedAt': '2021-04-07T10:00:28Z',
  'channelId': 'UC0xRMqPOyRNPTaL6BxhbCnQ',
  'title': '【実演】めんどくさい業務をプログラミング（Python）で自動化する過程をお見せします',
  'description': 'ビジネス戦闘力が圧倒的に向上するプログラミングスクール 『code4biz』  \u200d   ↓説明会はこちら↓ http://code4biz.jp/ ※ 締切 ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/bsq2YY-XAEo/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/bsq2YY-XAEo/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/bsq2YY-XAEo/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'いまにゅのプログラミング塾',
  'liveBroadcastContent': 'none',
  'publishTime': '2021-04-07T10:00:28Z'}}

In [7]:
video_id = item['id']['videoId']

In [8]:
channel_id = item['snippet']['channelId']

In [9]:
import pandas as pd

In [10]:
items_id = []
items = response['items']
for item in items:
    item_id = {}
    item_id['video_id'] = item['id']['videoId']
    item_id['channel_id'] = item['snippet']['channelId']
    items_id.append(item_id)

df_video = pd.DataFrame(items_id)

In [12]:
df_video[:3]

,video_id,channel_id
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
1,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ
2,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw


In [15]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
developerKey=DEVELOPER_KEY)

def video_search(youtube, q='自動化', max_results=50):
    response = youtube.search().list(
    q=q,
    part="id,snippet",
    order='viewCount',
    type='video',
    maxResults=max_results
    ).execute()

    items_id = []
    items = response['items']
    for item in items:
        item_id = {}
        item_id['video_id'] = item['id']['videoId']
        item_id['channel_id'] = item['snippet']['channelId']
        items_id.append(item_id)

    df_video = pd.DataFrame(items_id)
    return df_video

In [16]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
developerKey=DEVELOPER_KEY)

df_video = video_search(youtube, q='Python 自動化', max_results=50)

In [17]:
df_video[:3]

,video_id,channel_id
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
1,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ
2,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw


In [18]:
channel_ids = df_video['channel_id'].unique().tolist()
channel_ids[:3]

['UC0xRMqPOyRNPTaL6BxhbCnQ',
 'UC8butISFwT-Wl7EV0hUK0BQ',
 'UCGlgXjYVoHLD86TQQ799WIw']

In [20]:
subscriber_list = youtube.channels().list(
    id=','.join(channel_ids),
    part="statistics",
    fields='items(id, statistics(subscriberCount))'
 ).execute()

In [21]:
subscriber_list['items'][:5]

[{'id': 'UCwhKqH0jDKm5vPOZ7WQ2R1A',
  'statistics': {'subscriberCount': '67800'}},
 {'id': 'UCazV3A3_1-Mtd6E_auw_ifg',
  'statistics': {'subscriberCount': '22100'}},
 {'id': 'UCXgGY0wkgOzynnHvSEVmE3A',
  'statistics': {'subscriberCount': '736000'}},
 {'id': 'UCNste53_VRtP6MGUfitOYQA', 'statistics': {'subscriberCount': '6110'}},
 {'id': 'UC0xRMqPOyRNPTaL6BxhbCnQ',
  'statistics': {'subscriberCount': '121000'}}]

In [34]:
subscribers = []
for item in subscriber_list['items']:
    subscriber={}
    subscriber['channel_id'] = item['id']
    subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
    subscribers.append(subscriber)

df_subscribers = pd.DataFrame(subscribers)

In [35]:
df_subscribers[:3]

,channel_id,subscriber_count
0,UCwhKqH0jDKm5vPOZ7WQ2R1A,67800
1,UCazV3A3_1-Mtd6E_auw_ifg,22100
2,UCXgGY0wkgOzynnHvSEVmE3A,736000


In [36]:
df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
df.head(3)

,video_id,channel_id,subscriber_count
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ,121000
1,KmqEyT7HznQ,UC0xRMqPOyRNPTaL6BxhbCnQ,121000
2,VRFfAeW30qE,UC0xRMqPOyRNPTaL6BxhbCnQ,121000


In [41]:
df_extracted = df[df['subscriber_count']<10000]
df_extracted

,video_id,channel_id,subscriber_count
33,YwIX8w9i2DM,UCbg0ptFI3yZ4Pe5vmUiEzzA,7330
40,YbGAUEjTKg4,UC6OrQk8WsnCOR1OezlUU9zQ,7000
41,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,7930
42,SPf_nINsCfw,UCGFxqqRaM97VjM5ToRfkoFg,6820
44,ku9xVrR2YK8,UCm7ZBVMISPFosktFoP421Lg,1930
47,mWPI8vI2scQ,UCkyrFlOF7I6U-IMUxum9HrA,6120
49,nCfAtMooIQQ,UCNste53_VRtP6MGUfitOYQA,6110


In [42]:
video_ids = df_extracted['video_id'].tolist()

In [47]:
video_list = youtube.videos().list(
    id=','.join(video_ids),
    part="snippet, statistics",
    fields='items(id, snippet(title),statistics(viewCount))'
 ).execute()

In [48]:
video_list['items'][:3]

[{'id': 'YwIX8w9i2DM',
  'snippet': {'title': 'Automate Excel with Python Tutorial (2020)'},
  'statistics': {'viewCount': '267773'}},
 {'id': 'YbGAUEjTKg4',
  'snippet': {'title': 'Web Automation using Python To Fill out A Form'},
  'statistics': {'viewCount': '106921'}},
 {'id': 'uaIUOQ47eP8',
  'snippet': {'title': 'Google IT Automation with Python Professional Certificate'},
  'statistics': {'viewCount': '86213'}}]

In [49]:
items = video_list['items']
videos_info = []
for item in items:
    video_info = {}
    video_info['video_id'] = item['id']
    video_info['title'] = item['snippet']['title']
    video_info['view_count'] = item['statistics']['viewCount']
    videos_info.append(video_info)
df_videos_info = pd.DataFrame(videos_info)

In [50]:
df_videos_info[:3]

,video_id,title,view_count
0,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),267773
1,YbGAUEjTKg4,Web Automation using Python To Fill out A Form,106921
2,uaIUOQ47eP8,Google IT Automation with Python Professional ...,86213


In [51]:
results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
results[:3]

,video_id,channel_id,subscriber_count,title,view_count
0,YwIX8w9i2DM,UCbg0ptFI3yZ4Pe5vmUiEzzA,7330,Automate Excel with Python Tutorial (2020),267773
1,YbGAUEjTKg4,UC6OrQk8WsnCOR1OezlUU9zQ,7000,Web Automation using Python To Fill out A Form,106921
2,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,7930,Google IT Automation with Python Professional ...,86213


In [52]:
results = results.loc[:, ['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]

In [53]:
results[:3]

,video_id,title,view_count,subscriber_count,channel_id
0,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),267773,7330,UCbg0ptFI3yZ4Pe5vmUiEzzA
1,YbGAUEjTKg4,Web Automation using Python To Fill out A Form,106921,7000,UC6OrQk8WsnCOR1OezlUU9zQ
2,uaIUOQ47eP8,Google IT Automation with Python Professional ...,86213,7930,UCEq0oQsS-voRnSWbcviIDGA


In [56]:
def get_results(df_video, threshold=5000):
    channel_ids = df_video['channel_id'].unique().tolist()

    subscriber_list = youtube.channels().list(
        id=','.join(channel_ids),
        part='statistics',
        fields='items(id, statistics(subscriberCount))'
    ).execute()

    subsribers = []
    for item in subscriber_list['items']:
        subscriber = {}
        if len(item['statistics']) > 0:
            subscriber['channel_id'] = item['id']
            subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
        else:
            subscriber['channel_id'] = item['id']
        subscribers.append(subscriber)

    df_subscribers = pd.DataFrame(subscribers)

    df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
    df_extracted = df[df['subscriber_count'] < threshold]
    
    video_ids = df_extracted['video_id'].tolist()
    videos_list = youtube.videos().list(
        id=','.join(video_ids),
        part='snippet, statistics',
        fields='items(id, snippet(title), statistics(viewCount))'
    ).execute()
    videos_info = []
    items = video_list['items']
    for item in items:
        video_info = {}
        video_info['video_id'] = item['id']
        video_info['title'] = item['snippet']['title']
        video_info['view_count'] = item['statistics']['viewCount']
        videos_info.append(video_info)
    df_videos_info = pd.DataFrame(videos_info)

    results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
    results = results.loc[:, ['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]
    return results
    


In [59]:
df_video = video_search(youtube, q='Python Excel', max_results=50)
results = get_results(df_video, threshold=10000)

In [60]:
results[:3]

,video_id,title,view_count,subscriber_count,channel_id
0,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),267773,7330,UCbg0ptFI3yZ4Pe5vmUiEzzA
1,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),267773,7330,UCbg0ptFI3yZ4Pe5vmUiEzzA
2,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),267773,7330,UCbg0ptFI3yZ4Pe5vmUiEzzA
